# Voorbereiding

In [9]:
import pandas as pd

In [58]:
def lees_bevolking_van_jaartal(jaartal_inlezen):
    bestandsnaam = 'Bevolking_' + str(jaartal_inlezen) + '.csv'
    inleesbestand = pd.read_csv(bestandsnaam, sep=';')
    inleesbestand.head()
    
    mannen = inleesbestand.drop(['Vrouwen', 'Totaal'], axis=1)
    mannen.rename(columns={'Mannen':'Aantal'}, inplace=True)
    mannen['Geslacht'] = "Mannen"
    mannen.head()
    
    vrouwen = inleesbestand.drop(['Mannen', 'Totaal'], axis=1)
    vrouwen.rename(columns={'Vrouwen':'Aantal'}, inplace=True)
    vrouwen['Geslacht'] = "Vrouwen"
    vrouwen.head()
    
    bevolking = pd.concat([mannen, vrouwen], ignore_index=True)
    
    bevolking['Jaartal'] = jaartal_inlezen

    return bevolking

In [60]:
totale_bevolking = lees_bevolking_van_jaartal(2021)

for jaartal in range(2022,2024):
    nieuwe_bevolking = lees_bevolking_van_jaartal(jaartal)
    totale_bevolking = pd.concat([totale_bevolking, nieuwe_bevolking], ignore_index=True)

In [62]:
totale_bevolking.shape

(3828, 5)

In [64]:
totale_bevolking.head()

,NIS code,Woonplaats,Aantal,Geslacht,Jaartal
0,1000,België,5677211,Mannen,2021
1,2000,Vlaams Gewest,3293617,Mannen,2021
2,3000,Waals Gewest,1784520,Mannen,2021
3,4000,Brussels Hoofdstedelijk Gewest,599074,Mannen,2021
4,10000,Provincie Antwerpen,931025,Mannen,2021


In [72]:
# wij willen alles behalve de volgende filters om de dataset te normalizeren !
    # str.find(value) == -1 => vind alles wat niet 'de value' is
    
filt1 = (totale_bevolking['Woonplaats'].str.find('België') == -1)   # alles behalve 'België'
filt2 = (totale_bevolking['Woonplaats'].str.find('Gewest') == -1)   # alles behalve 'Gewest'
filt3 = (totale_bevolking['Woonplaats'].str.find('Provincie') == -1)   # alles behalve 'Provincie'
filt4 = (totale_bevolking['Woonplaats'].str.find('Arrondissement') == -1)   # alles behalve 'Arrondissement'

In [84]:
# we willen enkel de rijen overhouden waar alle filters geldig zijn !!!
totale_bevolking = totale_bevolking.loc[filt1 & filt2 & filt3 & filt4]

In [86]:
totale_bevolking.head()

,NIS code,Woonplaats,Aantal,Geslacht,Jaartal
6,11001,Aartselaar,7169,Mannen,2021
7,11002,Antwerpen,265213,Mannen,2021
8,11004,Boechout,6587,Mannen,2021
9,11005,Boom,9337,Mannen,2021
10,11007,Borsbeek,5349,Mannen,2021


In [97]:
# we willen elke NIS-code linken aan een provincie
    # zo krijgen we een beter beeld van de totale informatie

def provincie_volgens_niscode(code):

    code = str(code)
    
    if code[0] == '1': return 'Antwerp'
    if code[0] == '2'  and code[1] == '1': return 'Brussel'
    if code[0] == '2'  and code[1] == '3': return 'Vlaams-Brabant'
    if code[0] == '2'  and code[1] == '4': return 'Vlaams-Brabant'
    if code[0] == '2'  and code[1] == '5': return 'Waals-Brabant'
    if code[0] == '3': return 'West-Vlaanderen'
    if code[0] == '4': return 'Oost-Vlaanderen'
    if code[0] == '5': return 'Henegouwen'
    if code[0] == '6': return 'Luik'
    if code[0] == '7': return 'Limburg'
    if code[0] == '8': return 'Luxemburg'
    if code[0] == '9': return 'Namen'

In [93]:
totale_bevolking['Provincie'] = totale_bevolking['NIS code'].apply(provincie_volgens_niscode)

In [95]:
totale_bevolking.head()

,NIS code,Woonplaats,Aantal,Geslacht,Jaartal,Provincie
6,11001,Aartselaar,7169,Mannen,2021,Antwerp
7,11002,Antwerpen,265213,Mannen,2021,Antwerp
8,11004,Boechout,6587,Mannen,2021,Antwerp
9,11005,Boom,9337,Mannen,2021,Antwerp
10,11007,Borsbeek,5349,Mannen,2021,Antwerp


In [103]:
totale_bevolking = totale_bevolking.reset_index()

In [105]:
totale_bevolking.head()

,NIS code,Woonplaats,Aantal,Geslacht,Jaartal,Provincie
0,11001,Aartselaar,7169,Mannen,2021,Antwerp
1,11002,Antwerpen,265213,Mannen,2021,Antwerp
2,11004,Boechout,6587,Mannen,2021,Antwerp
3,11005,Boom,9337,Mannen,2021,Antwerp
4,11007,Borsbeek,5349,Mannen,2021,Antwerp


# Analyze

### Hoeveel verschillende NIS-codes bestaan er?

In [115]:
codes = totale_bevolking['NIS code'].nunique()
codes

581

### Wat is het aantal inwoners van heel België in het jaar 2023

In [131]:
filt = (totale_bevolking['Jaartal'] == 2023)

In [141]:
totaal_belgie = totale_bevolking.loc[filt, 'Aantal'].sum()

In [143]:
totaal_belgie

11697557

### Wat is de evolutie van het aantal inwoners van heel België over de jaren heen?

In [164]:
for jaartal in range(2021, 2024):
    filt = (totale_bevolking['Jaartal'] == jaartal)
    totaal_belgie = totale_bevolking.loc[filt, 'Aantal'].sum()
    print(f'{jaartal} - {totaal_belgie}')

2021 - 11521238
2022 - 11584008
2023 - 11697557


### Wat is het totaal aantal inwoners per provincie, per jaar?

In [171]:
inwoners_per_jaar_per_provincie = totale_bevolking.groupby(['Jaartal', 'Provincie'])['Aantal'].sum()

In [187]:
inwoners = inwoners_per_jaar_per_provincie.to_frame('Totaal_Inwoners').sort_values(['Jaartal', 'Totaal_Inwoners'], ascending = False)

In [217]:
inwoners

Totaal_Inwoners
Jaartal Provincie                       
2023    Antwerp                  1910952
        Oost-Vlaanderen          1561316
        Henegouwen               1356895
        Brussel                  1241175
        West-Vlaanderen          1220026
        Vlaams-Brabant           1187483
        Luik                     1115518
        Limburg                   895030
        Namen                     502261
        Waals-Brabant             412934
        Luxemburg                 293967
2022    Antwerp                  1886609
        Oost-Vlaanderen          1543865
        Henegouwen               1351127
        Brussel                  1222637
        West-Vlaanderen          1209011
        Vlaams-Brabant           1173440
        Luik                     1110989
        Limburg                   885951
        Namen                     499454
        Waals-Brabant             409782
        Luxemburg                 291143
2021    Antwerp                  1875524
        Oost-Vlaanderen          1531745
        Henegouwen               1345947
        Brussel                  1219970
        West-Vlaanderen          1203312
        Vlaams-Brabant           1162084
        Luik                     1109067
        Limburg                   880397
        Namen                     497073
        Waals-Brabant             407397
        Luxemburg                 288722

### Welke Provincie heeft de meeste inwoners per jaar?

In [215]:
for jaartal in range(2021, 2024):
    max_provincie = inwoners_per_jaar_per_provincie.loc[jaartal].max()
    max_inwoners = inwoners_per_jaar_per_provincie.loc[jaartal].max()
    print(f'{jaartal} - {max_per_jaar}')

2021 - 1875524
2022 - 1886609
2023 - 1910952


Ik wil een duidelijkere representatie van de conclusie !!!

In [242]:
for jaartal in range(2021, 2024):
    # .max() geeft enkel the waardes terug
        # => om de overeenkomende id eruit te halen gebruiken we .idxmax() !!!
    
    max_provincie = inwoners_per_jaar_per_provincie.loc[jaartal].idxmax()
    max_inwoners = inwoners_per_jaar_per_provincie.loc[jaartal].max()
    print(f'{jaartal}: {max_provincie} = {max_inwoners}')

2021: Antwerp = 1875524
2022: Antwerp = 1886609
2023: Antwerp = 1910952


### Wat is het hoogste aantal inwoners dat elke provincie ooit heeft behaald?

In [283]:
# .unique() geeft ons alle unieke waardes van de geselecteerde kolom !!!
unieke_provincies = totale_bevolking['Provincie'].unique()
provincies = [provincie for provincie in unieke_provincies]

In [279]:
provincies

['Antwerp',
 'Brussel',
 'Vlaams-Brabant',
 'Waals-Brabant',
 'West-Vlaanderen',
 'Oost-Vlaanderen',
 'Henegouwen',
 'Luik',
 'Limburg',
 'Luxemburg',
 'Namen']

In [333]:
for provincie in provincies:
    filt = (totale_bevolking['Provincie'] == provincie)
    som_jaar = totale_bevolking.loc[filt].groupby('Jaartal')['Aantal'].sum()
    
    jaar = som_jaar.idxmax()
    max_inwoners = som_jaar.max()

    print(f'{provincie} ({jaar}): {max_inwoners}')

Antwerp (2023): 1910952
Brussel (2023): 1241175
Vlaams-Brabant (2023): 1187483
Waals-Brabant (2023): 412934
West-Vlaanderen (2023): 1220026
Oost-Vlaanderen (2023): 1561316
Henegouwen (2023): 1356895
Luik (2023): 1115518
Limburg (2023): 895030
Luxemburg (2023): 293967
Namen (2023): 502261


### In welke provincies wonen er meer mannen dan vrouwen?

In [385]:
totale_bevolking.head()

,NIS code,Woonplaats,Aantal,Geslacht,Jaartal,Provincie
0,11001,Aartselaar,7169,Mannen,2021,Antwerp
1,11002,Antwerpen,265213,Mannen,2021,Antwerp
2,11004,Boechout,6587,Mannen,2021,Antwerp
3,11005,Boom,9337,Mannen,2021,Antwerp
4,11007,Borsbeek,5349,Mannen,2021,Antwerp


In [427]:
groep_geslacht = totale_bevolking.groupby(['Jaartal', 'Provincie', 'Geslacht'])['Aantal'].sum()

In [429]:
groep_geslacht

Jaartal  Provincie        Geslacht
2021     Antwerp          Mannen      931025
                          Vrouwen     944499
         Brussel          Mannen      599074
                          Vrouwen     620896
         Henegouwen       Mannen      655872
                                       ...  
2023     Vlaams-Brabant   Vrouwen     604956
         Waals-Brabant    Mannen      200139
                          Vrouwen     212795
         West-Vlaanderen  Mannen      604831
                          Vrouwen     615195
Name: Aantal, Length: 66, dtype: int64

In [461]:
# alle jaartallen afgaan
for jaartal in range(2021, 2024):

    print(('-')*20)
    print(jaartal)
    meer_mannen = []
    
    # alle provincies per jaartal afgaan
    for provincie in provincies:
    
        # .get(index) => door de indexen te navigeren kunnen we specifieke waardes selecteren !!!
        aantal_mannen = groep_geslacht.get(key = (jaartal, provincie, 'Mannen'), default = 0)
        aantal_vrouwen = groep_geslacht.get(key = (jaartal, provincie, 'Vrouwen'), default = 0)
        
        if aantal_mannen > aantal_vrouwen:
            meer_mannen.append(provincie)

    print(meer_mannen)
    print(('-')*20)
    print()

--------------------
2021
[]
--------------------

--------------------
2022
[]
--------------------

--------------------
2023
[]
--------------------

